In [ ]:
import pandas

import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import multipolygon, polygon, Polygon, MultiPolygon

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pandas.options.display.max_columns = 100


### Notebook objectives

* Load in data of 2019 General election data (2024) when available
* Understand how to plot a shape file
* How to link the Results to the consituency map shape file

## Data Loading

* Created a Data drive to store the files. Able to read in the voting results data and parliment boundaries shape data and basically plot it.
* The results data shows consituency id (ONS ID), as well as winning party, type of result (Gain vs Hold) and vote breakdown.
* loading in the Geopandas data shows that the table is GeoDataFrame type.
* The UK is plotted out which shows that the plotting (and adding) of each constituency to the plot forms a complete picture

In [ ]:
# Read in the data, first row is table header
data = "data/ge_2019_results.xlsx"
results = pandas.read_excel(data, header=1)

map_data = "data/PCON_DEC_2019_UK_BFC.shp"
boundaries_map = gpd.read_file(map_data)
results.head()
boundaries_map.head()
boundaries_map.info()
boundaries_map.plot()


### Plotting Plymouth Data

* Decided to plot plymouth data to investigate the plotting further
* **NOTE** that i have manually assigned colours for visual representation at this point


In [ ]:
color_map = {349:"red",
              350:"blue"}

ply_df = boundaries_map.loc[boundaries_map["PCON19NM"].str.contains("Plymouth")]
ply_df

#ply_df.info()
fig, ax = plt.subplots(figsize=(10,6))

for idx, row in ply_df.iterrows():
    row_df = gpd.GeoDataFrame(row.to_frame().T)
    #row_df.info()
    _ = row_df.plot(ax=ax, 
            color=color_map[idx],
            label=row_df["PCON19NM"].values.tolist()[0] )
    
    _ = row_df.apply(lambda x: ax.annotate(text=x['PCON19NM'], 
                                       xy=x.geometry.centroid.coords[0], 
                                       fontsize = 12,
                                       backgroundcolor = '0.75',
                                       ha='center'), axis=1);



### Joining data together

* Looked ID comparison and name comparison to see how to join the data.
* ID data appears to match one-to-one.
* The name data appears not quite match with some values appearing in 1 dataframe but not the other, this appears to be punctuation related.

In [ ]:
[x for x in results["ONS ID"].values if x not in boundaries_map["PCON19CD"].values]
[x for x in results["Constituency name"].values if x not in boundaries_map["PCON19NM"].values]
[x for x in boundaries_map["PCON19NM"].values if x not in results["Constituency name"].values]

In [ ]:
all_data_df = (results.merge(boundaries_map,
                            how="inner",
                            left_on="ONS ID",
                            right_on="PCON19CD")
    .sort_values(["Constituency name"], ignore_index=True)
)                            
all_data_df.head(3)

### Plotting all data (ex NI) 

WE want to plat the UK data with the winning parlimentary colours

In [ ]:
all_data_df["First party"].value_counts()
all_data_df = gpd.GeoDataFrame(all_data_df)

In [ ]:
colour_map = {"Lab":"red",
              "Con":"blue",
              "SNP":"yellow",
              "LD":"orange",
              "PC":"brown",
              "BRX":"teal",
              "Grean":"green",
              "DUP":"purple",
              "SF":"purple",
              "SDLP":"purple",
              "Spk":"purple",
              "APNI":"purple",
              }
all_data_df["colour"] = all_data_df["First party"].replace(colour_map)

In [ ]:
#ply_df.info()
fig, ax = plt.subplots(figsize=(25,20))

all_data_df.plot(ax=ax, 
    color=all_data_df["colour"]
    )
